In [ ]:
baseurl='https://graph.facebook.com/v2.4/'
userid=''
res_url=baseurl+userid+'?fields=posts{full_picture,message,message_tags,likes{name},created_time}'
comm_url = baseurl+userid+'?fields=posts{comments}'
access_token=''

In [ ]:
import requests
import json
import os
import re

#抓取json(一開始查詢的連接)
def url_res(url,access_token):
    header={'Authorization':'Bearer {}'.format(access_token)}
    res = requests.get(url,headers=header)
    data = res.json()
    return data['posts']

#讀取json(next的連接)
def url_res_noposts(url):
    res = requests.get(url)
    data = res.json()
    return data

#不知道為甚麼nexturl抓出來只有一半,所以就加上access_token
def next_url(url,access_token):
    return url+'&access_token='+access_token

#將pic ture抓取下來,依照頁數分類
def img_parser(data):
    if not os.path.exists('pic'):
        os.makedirs('pic')
    for data in  data['data']:
        if  'full_picture' in data:
            pic_url = data['full_picture']
            filename = data['id']
            m = re.search('(?P<picname>[\d_]+)',filename)
            if m:
                picname=m.group('picname')
            pic = open('./pic/{}.jpg'.format(picname),'wb')
            pic.writelines(requests.get(pic_url).content)
            pic.close()
            

#將message抓取
def message(data):
    if not os.path.exists('message'):
        os.makedirs('message')
    for data in data['data']:
        if 'message' in data:
            message = data['message']
            filename = data['id']
            m = re.search('(?P<messname>[\d_]+)',filename)
            if m:
                messname=m.group('messname')
            mess = open('./message/{}.txt'.format(messname),'w')
            mess.write(message.encode('utf8'))
            mess.close()

#抓取評論
def comments(data):
    if not os.path.exists('comments'):
        os.makedirs('comments')
    for ele in data['data']:
        commlist=[]
        if 'comments' in ele:
            filename = ele['id']
            for data in ele['comments']['data']:
                message = data['message']
                m = re.search('(?P<commname>[\d_]+)',filename)
                commlist.append(data['created_time'].encode('utf8')+'\n'+data['from']['name'].encode('utf8')+'\n'+data['message'].encode('utf8')+'\n'+'================================================='+'\n')
                if m:
                    commname=m.group('commname') 
            comm = open('./comments/{}.txt'.format(commname),'w')
            for ele in commlist:
                comm.write(ele)
            comm.close()

#判斷是否還有下一頁
def nextfrompage(data):
    data=data['paging']
    next_page=data['next']
    if next_page is None:
        return None
    else:
        return True

#抓取下一頁連接
def nexturl(data):
    data=data['paging']
    next_page=data['next']
    return next_page

#主程式
def main():
    
    data=url_res(res_url,access_token)
    comm_data=url_res(comm_url,access_token)
    next_page=nextfrompage(data)
    
    while(next_page):
        img_parser(data)
        message(data)
        comments(comm_data)
        #判斷下一頁及抓取下一頁連接
        next_page=nextfrompage(data)
        normal_next_url=nexturl(data)
        comm_next_url=nexturl(comm_data)
        #取得下一頁的json
        data=url_res_noposts(next_url(normal_next_url,access_token))
        comm_data=url_res_noposts(next_url(comm_next_url,access_token))
        
#print 'done'

if __name__ == '__main__':
    main()